OK, I'm busy at the moment, so I'll do this as quickly as possible.

# Part 1

I don't know how dense the program space is, so I'll use a defaultdict. A list would be better, but we don't know how big the list is going to get. Probably easy enough to work out, but I can't be bothered. Mind you, there's probably some sort of automatically expanding list in python, but I don't know what it is.

In [1]:
import collections

To get the initial state, parse the input into a list of integers:

In [2]:
test_initial_state='1,9,10,3,2,3,11,0,99,30,40,50'

initial_input=collections.defaultdict(int)

for (i, v) in enumerate([int(ix.strip()) for ix in test_initial_state.split(',')]):
    initial_input[i]=v
    
initial_input

defaultdict(int,
            {0: 1,
             1: 9,
             2: 10,
             3: 3,
             4: 2,
             5: 3,
             6: 11,
             7: 0,
             8: 99,
             9: 30,
             10: 40,
             11: 50})

Helpful to have an input parser for the test cases:

In [3]:
def parse_input(str_in):
    initial_input=collections.defaultdict(int)
    
    for (i, v) in enumerate([int(ix.strip()) 
                             for ix in str_in.split(',')]):
        initial_input[i]=v
    
    return initial_input

In [4]:
parse_input('1,9,10,3,2,3,11,0,99,30,40,50')

defaultdict(int,
            {0: 1,
             1: 9,
             2: 10,
             3: 3,
             4: 2,
             5: 3,
             6: 11,
             7: 0,
             8: 99,
             9: 30,
             10: 40,
             11: 50})

OK, now something for the program runner:

In [14]:
def run_program(program_dict_in, verbose=False):
    program=program_dict_in.copy()
    state=0
    while program[state]!=99:
        if program[state]==1:
            program[program[state+3]]=program[program[state+1]]+program[program[state+2]]
            state+=4
        elif program[state]==2:
            program[program[state+3]]=program[program[state+1]]*program[program[state+2]]
            state+=4
        if verbose:
            l=[0]*max(program.values())
            for (k, v) in program.items():
                l[k]=v
            print(l)
            
    return program
        
        

In [16]:
run_program(parse_input('1,0,0,0,99'))

defaultdict(int, {0: 2, 1: 0, 2: 0, 3: 0, 4: 99})

In [17]:
run_program(parse_input('2,3,0,3,99'))

defaultdict(int, {0: 2, 1: 3, 2: 0, 3: 6, 4: 99})

In [16]:
run_program(parse_input('1,0,0,0,99'))

defaultdict(int, {0: 2, 1: 0, 2: 0, 3: 0, 4: 99})

In [18]:
run_program(parse_input('2,4,4,5,99,0'))

defaultdict(int, {0: 2, 1: 4, 2: 4, 3: 5, 4: 99, 5: 9801})

In [19]:
run_program(parse_input('1,1,1,4,99,5,6,0,99'))

defaultdict(int, {0: 30, 1: 1, 2: 1, 3: 4, 4: 2, 5: 5, 6: 6, 7: 0, 8: 99})

That all seems to be working OK. So now use the puzzle input:

In [20]:
with open('inputs/day_02.txt') as fIn:
    puzzle_input_str=fIn.read()
puzzle_input_str

'1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,13,1,19,1,19,10,23,1,23,6,27,1,6,27,31,1,13,31,35,1,13,35,39,1,39,13,43,2,43,9,47,2,6,47,51,1,51,9,55,1,55,9,59,1,59,6,63,1,9,63,67,2,67,10,71,2,71,13,75,1,10,75,79,2,10,79,83,1,83,6,87,2,87,10,91,1,91,6,95,1,95,13,99,1,99,13,103,2,103,9,107,2,107,10,111,1,5,111,115,2,115,9,119,1,5,119,123,1,123,9,127,1,127,2,131,1,5,131,0,99,2,0,14,0'

The puzzle needs us to change two of the values, and return the final value in position 0:

In [21]:
# Parse the input

input_program=parse_input(puzzle_input_str)

# Change the initial values:

input_program[1]=12
input_program[2]=2

# Run the program, and return the value at position 0

run_program(input_program)[0]

5866714

Correct!

# Part 2

I have no wish to spend more time on this than necessary, so I'll do a brute force thing. I suspect that some of the inputs won't terminate (otherwise it wouldn't be very interesting...), so I'll need to check for that.

Let's alter the interpreter so that it returns False if:
- we go to an error state so that the command isn't 1 or 2, or
- we go to a state that's already been visited, or
- the program gets too big.

In [41]:
def run_program(program_dict_in, max_size=1000, verbose=False):
    program=program_dict_in.copy()
    state=0
    
    def program_to_string(p_in):
        l=[0]*(max(p_in.keys())+1)
        for (k, v) in p_in.items():
            l[k]=v
        return(l)
        
    visited_states_ls=[(state, program_to_string(program))]
    if verbose:
        print(str(visited_states_ls[0]))
    
    while program[state]!=99:
        if program[state] not in [1, 2, 99]:
            print('Bad command')
            return False
        elif max(program.keys())>max_size:
            print('Too big')
            return False
        elif program[state]==1:
            program[program[state+3]]=program[program[state+1]]+program[program[state+2]]
            visited_states_ls.append((state, program_to_string(program)))
            state+=4
        elif program[state]==2:
            program[program[state+3]]=program[program[state+1]]*program[program[state+2]]
            visited_states_ls.append((state, program_to_string(program)))
            state+=4
        if verbose:
            print(str((state, program_to_string(program))))
        if (state, program_to_string(program)) in visited_states_ls:
            print('Visited state: {}'.format((state, program_to_string(program))))            
            return False
            
            
    return program

In [44]:
p=parse_input('1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,13,1,19,1,19,10,23,1,23,6,27,1,6,27,31,1,13,31,35,1,13,35,39,1,39,13,43,2,43,9,47,2,6,47,51,1,51,9,55,1,55,9,59,1,59,6,63,1,9,63,67,2,67,10,71,2,71,13,75,1,10,75,79,2,10,79,83,1,83,6,87,2,87,10,91,1,91,6,95,1,95,13,99,1,99,13,103,2,103,9,107,2,107,10,111,1,5,111,115,2,115,9,119,1,5,119,123,1,123,9,127,1,127,2,131,1,5,131,0,99,2,0,14,0')
p[1]=12
p[2]=2
run_program(p, verbose=True)

(0, [1, 12, 2, 3, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 13, 1, 19, 1, 19, 10, 23, 1, 23, 6, 27, 1, 6, 27, 31, 1, 13, 31, 35, 1, 13, 35, 39, 1, 39, 13, 43, 2, 43, 9, 47, 2, 6, 47, 51, 1, 51, 9, 55, 1, 55, 9, 59, 1, 59, 6, 63, 1, 9, 63, 67, 2, 67, 10, 71, 2, 71, 13, 75, 1, 10, 75, 79, 2, 10, 79, 83, 1, 83, 6, 87, 2, 87, 10, 91, 1, 91, 6, 95, 1, 95, 13, 99, 1, 99, 13, 103, 2, 103, 9, 107, 2, 107, 10, 111, 1, 5, 111, 115, 2, 115, 9, 119, 1, 5, 119, 123, 1, 123, 9, 127, 1, 127, 2, 131, 1, 5, 131, 0, 99, 2, 0, 14, 0])
(4, [1, 12, 2, 3, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 13, 1, 19, 1, 19, 10, 23, 1, 23, 6, 27, 1, 6, 27, 31, 1, 13, 31, 35, 1, 13, 35, 39, 1, 39, 13, 43, 2, 43, 9, 47, 2, 6, 47, 51, 1, 51, 9, 55, 1, 55, 9, 59, 1, 59, 6, 63, 1, 9, 63, 67, 2, 67, 10, 71, 2, 71, 13, 75, 1, 10, 75, 79, 2, 10, 79, 83, 1, 83, 6, 87, 2, 87, 10, 91, 1, 91, 6, 95, 1, 95, 13, 99, 1, 99, 13, 103, 2, 103, 9, 107, 2, 107, 10, 111, 1, 5, 111, 115, 2, 115, 9, 119, 1, 5, 119, 123, 1, 123, 9, 127, 1, 127, 2,

defaultdict(int,
            {0: 5866714,
             1: 12,
             2: 2,
             3: 2,
             4: 1,
             5: 1,
             6: 2,
             7: 3,
             8: 1,
             9: 3,
             10: 4,
             11: 3,
             12: 1,
             13: 5,
             14: 0,
             15: 3,
             16: 2,
             17: 13,
             18: 1,
             19: 60,
             20: 1,
             21: 19,
             22: 10,
             23: 64,
             24: 1,
             25: 23,
             26: 6,
             27: 66,
             28: 1,
             29: 6,
             30: 27,
             31: 68,
             32: 1,
             33: 13,
             34: 31,
             35: 73,
             36: 1,
             37: 13,
             38: 35,
             39: 78,
             40: 1,
             41: 39,
             42: 13,
             43: 83,
             44: 2,
             45: 43,
             46: 9,
             47: 249,
     

OK, so we've got (I hope) the working interpreter now. Let's go through all the possible pairings and see what happens:

In [48]:
with open('inputs/day_02.txt') as fIn:
    puzzle_input_str=fIn.read()
    
for i in range(100):
    print(i, end=' ')
    for j in range(100):
        p=parse_input(puzzle_input_str)
        p[1]=i
        p[2]=j
        o=run_program(p)
        if o[0]==19690720:
            print
            print((i, j))
            
        

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 (52, 8)
53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

5208, then. Damn it, could have done that in a couple of minutes. Serves me right for writing defensive code...